In [1]:
!pip install folium
import requests

import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0
import folium 

from collections import Counter
from bs4 import BeautifulSoup


     |████████████████████████████████| 102kB 7.3MB/s ta 0:00:011


In [2]:
CLIENT_ID # Foursquare ID
CLIENT_SECRET  # Foursquare Secret
VERSION = '20180604'
LIMIT = 50
radius = 500

<h3> Get 30 most popular venue categories from each location in Foursquare API </h3>

In [3]:
locations = ["New York City, NY", "San Francisco, CA", "Chicago, IL", "Boston, MA", "Philadelphia, PA"]
popular_venue_data = {}
for location in locations:
    location_data = {}
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&near={}&v={}&radius={}&limit={}&sortByPopularity=1'.format(CLIENT_ID, CLIENT_SECRET, location, VERSION, radius, LIMIT)
    results = requests.get(url).json()
    for venue in results['response']['groups'][0]['items']:
        venue_type = venue['venue']['categories'][0]['name']
        if venue_type in location_data:
            location_data[venue_type] += 1
        else:
            location_data[venue_type] = 1
    popular_venue_data[location]=location_data


<h2> Get 5 most popular venues and insert data into Pandas DataFrame </h2>

<h6> Sorted in order from Most Popular (1) to Least Popular (5)</h6>

In [4]:
data = {'City':[], 1:[], 2:[], 3:[], 4:[], 5:[]}
for loc in locations:
    popular_venue_data[loc] = [i[0] for i in Counter(popular_venue_data[loc]).most_common(5)]
    data['City'].append(loc)
    for i in range(1,6):
        data[i].append(popular_venue_data[loc][i-1])
popular_venue_df=pd.DataFrame(data)
popular_venue_df["City"].replace({"New York City, NY":"NYC", "San Francisco, CA":"San Francisco", "Chicago, IL":"Chicago", "Boston, MA":"Boston", "Philadelphia, PA":"Philadelphia"}, inplace=True)

<H1> DataFrame of the Population Density in Each Location </H1>

<h3> Scraped from governing.com using BeautifulSoup and data entered into a Pandas DF</h3>

In [5]:
res = requests.get("https://www.governing.com/gov-data/population-density-land-area-cities-map.html")
soup = BeautifulSoup(res.content, 'lxml')
tables = soup.find_all('table')
locations = ["New York, New York", "San Francisco", "Chicago", "Boston", "Philadelphia"]
for table in tables:
    table_rows = table.find_all('tr')
    res = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        for elem in locations:
            if len(row)>1 and elem in row[0] :
                res.append(row)
                locations.remove(elem)
                break
                
population_density_df = pd.DataFrame(res, columns=['City', 'Population Density (Persons/Square Mile)', '2016 Population', 'Land Area (Square Miles)'])
population_density_df["City"].replace({"New York, New York":"NYC", "San Francisco, California":"San Francisco", "Chicago, Illinois":"Chicago", "Boston, Massachusetts":"Boston", "Philadelphia, Pennsylvania":"Philadelphia"}, inplace=True)

<H1> DataFrame of Median Home Price in Each Location </H1>

<h3> Scraped from kiplinger.com using BeautifulSoup and data entered into a Pandas DF</h3>

In [6]:
res = requests.get("https://www.kiplinger.com/article/real-estate/T010-C000-S002-home-price-changes-in-the-100-largest-metro-areas.html")
soup = BeautifulSoup(res.content, 'lxml')
tables = soup.find_all('table')
locations = ["New York, N.Y.", "San Francisco", "Chicago", "Boston", "Philadelphia"]
for table in tables:
    table_rows = table.find_all('tr')
    res = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        for elem in locations:
            if len(row)>1 and elem in row[0] :
                res.append(row)
                locations.remove(elem)
                break
                
home_prices_df = pd.DataFrame(res, columns=['Metro Area', 'Median Home Price($)', '% Change 1 Year', '% Change Since Peak', "% Change Since Bottom", "Affordability Index"])
home_prices_df["Metro Area"].replace({"New York, N.Y.-N.J.":"NYC", "San Francisco, Calif.":"San Francisco", "Chicago, Ill.":"Chicago", "Boston, Mass.":"Boston", "Philadelphia, Pa.":"Philadelphia"}, inplace=True)


<H1> DataFrame of the School District Size in Each Location </H1>

<h3> Scraped from wikipedia.com using BeautifulSoup and data entered into a Pandas DF</h3>

In [7]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_the_largest_school_districts_in_the_United_States_by_enrollment")
soup = BeautifulSoup(res.content, 'lxml')
tables = soup.find_all('table')
locations = ["New York City", "San Francisco", "Chicago", "Boston", "Philadelphia"]
for table in tables:
    table_rows = table.find_all('tr')
    res = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        for elem in locations:
            if len(row)>1 and elem in row[2] :
                res.append(row)
                locations.remove(elem)
                break
                
school_df = pd.DataFrame(res, columns=['Rank', 'vs 2014', 'School District', "State/Territory", "Students in Autumn of 2016"])
school_df["School District"].replace({"New York City":"NYC", "San Francisco Unified":"San Francisco", "City of Chicago (SD 299)":"Chicago", "Philadelphia City":"Philadelphia"}, inplace=True)


<h1> Compile All Data from previous dfs into a single df </h1>

In [8]:
data = {'City':[], "School District Size":[], "Population Density":[], "Median Home Price ($)":[], "Most Popular Venue":[], "Venue 2":[], "Venue 3":[], "Venue 4":[], "Venue 5":[]}
locations = ["NYC", "San Francisco", "Chicago", "Philadelphia", 'Boston']

for loc in locations:
    data['City'].append(loc)
    data['School District Size'].append(school_df['Students in Autumn of 2016'][school_df['School District']== loc].iloc[0])
    data['Population Density'].append(population_density_df['Population Density (Persons/Square Mile)'][population_density_df['City']==loc].iloc[0])
    data['Median Home Price ($)'].append(home_prices_df['Median Home Price($)'][home_prices_df['Metro Area']==loc].iloc[0])
    data['Most Popular Venue'].append(popular_venue_df[1][popular_venue_df['City']==loc].iloc[0])
    data['Venue 2'].append(popular_venue_df[2][popular_venue_df['City']==loc].iloc[0])
    data['Venue 3'].append(popular_venue_df[3][popular_venue_df['City']==loc].iloc[0])
    data['Venue 4'].append(popular_venue_df[4][popular_venue_df['City']==loc].iloc[0])
    data['Venue 5'].append(popular_venue_df[5][popular_venue_df['City']==loc].iloc[0])

overall_df = pd.DataFrame(data)
overall_df

,City,School District Size,Population Density,Median Home Price ($),Most Popular Venue,Venue 2,Venue 3,Venue 4,Venue 5
0,NYC,"984,462","28,211","418,000",American Restaurant,Gym / Fitness Center,Coffee Shop,Grocery Store,Hotel
1,San Francisco,"60,133","18,581","875,000",Coffee Shop,Café,Cocktail Bar,New American Restaurant,Food & Drink Shop
2,Chicago,"378,199","11,883","229,000",Mexican Restaurant,Train Station,Dive Bar,Donut Shop,Brewery
3,Philadelphia,"133,929","11,692","220,000",Hotel,Plaza,Grocery Store,Convenience Store,Coffee Shop
4,Boston,"53,640","13,943","421,000",American Restaurant,Historic Site,Coffee Shop,Pub,Market
